# Figures associated to vasculature

**You need to tun the notebook 2_ first!!!!**

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk
import itertools
import matplotlib.pylab as pylab

In [ ]:
from functions import adata_plot_KOvsWT, stat_annot_gene
from functions import dict_WT_KO_colors

In [ ]:
from datetime import date

today = date.today()
today = today.strftime("%Y_%m_%d")

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
params = {'figure.dpi': 200,
          'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large', 
         'axes.grid': False}
pylab.rcParams.update(params)
# mpl.style.use('default')

pd.set_option('display.max_columns', None)

## Functions and variables

In [ ]:
%store -r dict_WT_KO_colors
%store -r seed

## Load adata

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')
adata_vasc = adata_all_harmony[adata_all_harmony.obs['cell_type'].isin(['Lymphatic', 'Endothelial', 'Perivascular cell $Il6^+$',
       'Perivascular cell $Inpp4b^+$', 'Schwann cell',])]

In [ ]:
sc.pp.filter_genes(adata_vasc, min_counts=1)

In [ ]:
sc.pp.pca(adata_vasc, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_vasc, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_vasc, random_state=seed, n_neighbors=int(len(adata_vasc) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_vasc, use_raw=False)

sc.pp.pca(adata_vasc, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_vasc, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_vasc, random_state=seed, n_neighbors=int(len(adata_vasc) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')

### Subpopulation annotation

To do the annotation we used the characterization made by Joost et al. 2020. There are some populations that are not appearing (Krt75+ to Krt18+) and we are not sure about their functional relationship

In [ ]:
dict_vasc = {    
                 'Endo 0 (vein)': ['Ackr1', 'Spint2', 'Selp', 'Aqp1', 'Fbln2', 'Ehd4', 'Sele', 'Vwf',  'Pdlim1', 'Tspan7', 'Lepr', 'Chp2', 'Grrp1', 'Fbn1', 'Marveld1', 'Cadm3', 'Nuak1', ],
                 'Endo 1 (capillary)': ['Prag1', 'Cyp1a1', 'Ccl2', 'Rgcc', 'Gdf15', 'Cd36', 'Rgs5', 'Pcdh1', 'Slc30a1', 'Tcf15', 'Pdgfrb', 'Cryab', 'Col6a1', 'Nrarp', 'Col3a1', 'Gclc', ],
    'Endo 2 (capillary)': ['Pald1', 'Nos2', 'Chst1', 'Col4a1', 'Rgcc', 'Adgrl2', 'Hspg2', 'Fscn1', 'Plekha1', 'Fam43a', 'Cxcr4', 'Sipa1', 'Afap1l1', 'Jcad', 'Smagp', 'Adamts7', 'Nrros', ],
    'Endo 3': ['Cp', 'Pf4', 'Fzd6', 'Cxcl2', 'Selp', 'Csf1', 'Mgp', 'Pfkfb3', 'Igfbp5', 'Sdc4', 'Sele', 'F2r', 'Rnd1', 'Wnt5b', ],
    'Endo 4 (artery)': ['Clu', 'Edn1', 'Fbln5', 'Cst3', 'Fbln2', 'Ltbp4', 'Eln', 'Ptprb', 'Fstl1', 'Stmn2', 'Ly6c1', 'Sema3g', 'Podxl', 'Ly6a', 'Htra1', 'S100a4', 'Ace', ],
    'Endo 5': ['Tyrobp', 'Ccl3', 'Fcer1g', 'Srgn', 'Ifitm1', 'Ccl6', 'Alox5ap', 'Lilrb4a', 'Lgals3', 'Cd52', 'Ccl4', 'Rgs1', 'Samsn1', 'Osm', 'Taldo1', 'Lat2', 'Mcpt8', 'Slpi', 'Cd53', ],
    'Peri 0': ['Selenbp1', 'Galk1', 'Tnfrsf9', 'Pkm', 'Pgk1', 'Aldoa', 'Bsg', 'Higd1a', 'Tpi1', 'Ankrd37', 'P4ha2', 'Bnip3l', 'Pfkp', 'Cox7a1', 'Ppp1r3c', 'Kbtbd11', 'Hk1', ],
    'Peri 1': ['Ccl7', 'Ccl11', 'Irak3', 'Procr', 'Bach1', 'Crispld2', 'Cnbp', 'Crem', 'Uap1', 'Eprs', 'Vat1', 'Mpp7', 'Nsd3', 'Dot1l', 'Ccl2', 'Cxcl1', 'Cxcl2', 'Nfkb1', 'Plpp3', 'Il6', 'Steap4', 'Adamts4', 'Tnc', 'Pkdcc', ],
    'Peri 2': ['Vcam1', 'Cgnl1', 'Tgfbi', 'Cd44', 'Snhg1', 'Flnb', 'Hivep2', 'Lhfpl2', 'Arl5b', 'Mcc', 'Cd34', 'Tjp2', 'Gja1', ],
    'Peri 3': ['Nrep', 'Inpp4b', 'Dlk1', 'Sept4', 'Kitl', 'Cyp4b1', 'Abcc9', 'Mgp', 'Col3a1', 'Ano1', 'Marcks', 'Atp1b2', 'Thy1', 'Tppp3', 'Cygb', 'Mrvi1', 'Adap2', ],
    'Peri 4': ['Actg2', 'Nexn', 'Crlf1', 'Tpm2', 'Acta2', 'Myl6', 'Tagln', 'Flna', 'Mustn1', 'Eln', 'Mylk', 'Ndufa4', 'Hcfc1r1', 'Ccdc107', 'Cnn1', 'Dmpk', 'Smtn', ],
    'Lymph 0': ['Fndc1', 'Cavin2', 'Adgrg3', 'Ccl21a', 'Lyve1', 'Gng11', 'Mmrn1', 'Flt4', 'Slc45a3', 'Ramp2', 'Thy1', 'Reln', 'Ptpn18', 'Olfm1', 'Mmrn1', 'Fgl2', 'Prox1', ],
    'Lymph 1': ['Gdf15', 'Pgf', 'Dtx1', 'Nfat5', 'Ugcg', 'Cpne2', 'Cd24a', 'Neo1', 'Nupr1', 'Slc38a4', 'Adarb1', 'Tbc1d4', 'Plaur', 'Ahnak2', 'Retreg2', 'Atp6v1h',  'Olfm1', 'Mmrn1', 'Fgl2', 'Prox1', ],
    'Schwann cell': ['Mpz', 'Cryab', 'Mbp', 'Plp1', 'Sema3b', 'Egfl8', 'Mal', 'Limch1', 'Gatm', 'Dbi', 'Kcna1', 'Cnp', 'Prx', 'Ncmap', 'Cadm4', 'Plekha4', 'Cldn19', ]
                  }

In [ ]:
dict_vasc_colors = {'Endo 0 (vein)': "#8E0C6F",
                    'Endo 1 (capillary)': "#AE54B8",
                    'Endo 2 (capillary)': "#4D0765",
                    'Endo 3': "#20668D",
                    'Endo 4 (artery)': "#229EB2",
                    'Endo 5': "#77ECF1",
                    'Lymph 0': "#FF5599",
                    'Lymph 1': "#D40055",
                    'Peri 0': "#D3BC5F",
                    'Peri 1': "#A5901A",
                    'Peri 2': "#A05A2C",
                    'Peri 3': "#702A17",
                    'Peri 4': "#4E1302",
                    'Schwann cell': "#FF9955"}

In [ ]:
sc.tl.leiden(adata_vasc, resolution=4)
assign_cats(adata_vasc, dict_cats=dict_vasc, min_score=0.4, quantile_gene_sel=0.65, key_added='subtype')

adata_vasc.uns['subtype_colors'] = list(dict_vasc_colors.values())

In [ ]:
sc.tl.umap(adata_vasc, min_dist=0.85)
sc.tl.leiden(adata_vasc, resolution=2)


# Markers extracte from Kalucka et al. 2020 (mmc5)

sc.pl.umap(adata_vasc, color=['leiden', 'batch', 'subtype'])
sc.pl.umap(adata_vasc, color=['Clu', 'Crip1', 'Fbln2', 'Gja4', 'Hey1', 'Mecom', 'Sat1', 'Sema3g', 'Sox17', 'Tm4sf1', 'Tsc22d1',], cmap=magma, legend_loc='on data') # Artery
sc.pl.umap(adata_vasc, color=['Car4', 'Cd200', 'Cd300lg', 'Gpihbp1', 'Kdr', 'Rgcc', 'Sgk1', 'Sparc',], cmap=magma, legend_loc='on data') # Capillary
sc.pl.umap(adata_vasc, color=['Apoe', 'Bgn', 'Ctla2a', 'Icam1', 'Il6st', 'Ptgs1', 'Tmsb10', 'Vcam1', 'Vwf', ], cmap=magma, legend_loc='on data') # Vein

## UMAP of fibroblast subpopulations and batch

In [ ]:
sc.pl.umap(adata_vasc[adata_vasc.obs['batch'].isin(['KO1', 'KO2'])] , color=['batch', 'subtype'], ncols=2, frameon=False, cmap=magma, size=35)
sc.pl.umap(adata_vasc[adata_vasc.obs['batch'].isin(['WT1', 'WT2'])] , color=['batch', 'subtype'], ncols=2, frameon=False, cmap=magma, size=35)

## Proportion of cell types within WT and KO

In [ ]:
adata_plot_KOvsWT(adata_vasc, list(dict_vasc.keys()), col_cell_type='subtype')

## Table with markers and p-values

In [ ]:
sc.tl.rank_genes_groups(adata_vasc, groupby='subtype')
names = pd.DataFrame(adata_vasc.uns['rank_genes_groups']['names'])
pvals = pd.DataFrame(adata_vasc.uns['rank_genes_groups']['pvals_adj'])

In [ ]:
# Create a DataFrame with 30 most important genes
list_dfs = []
for subtype, genes in dict_vasc.items():
    df = pd.DataFrame({f'{subtype} genes': names[subtype], f'p-value {subtype}': pvals[subtype]}, dtype=object)
    df_short = df[df[f'{subtype} genes'].isin(genes)].sort_values(by=f'p-value {subtype}', ascending=True).iloc[:30].reset_index(drop=True)
    df[f'p-value {subtype}'] = df[f'p-value {subtype}'].apply(lambda x: '%.3E' % x).values.tolist()
    list_dfs.append(df_short)

In [ ]:
df_all_celltypes = pd.concat(list_dfs, axis=1)
df_all_celltypes.to_csv(f'results/{today}-markers_4_pops.csv', index=False)
df_all_celltypes

## Dotplot with markers of fibroblast subtypes

In [ ]:
sc.pl.dotplot(adata_vasc, groupby='subtype', var_names=np.array([i[:3] for i in dict_vasc.values()]).ravel(), dendrogram=False, show=False, 
             categories_order=list(dict_vasc.keys()))

## UMAP highlighting the expression of specific markers

In [ ]:
list_genes = ['Hif1a', 'Vegfa', 'Vegfb']
sc.pl.umap(adata_vasc, color=['batch',  'subtype',], ncols=3, frameon=False, cmap=magma)
sc.pl.umap(adata_vasc, color=list_genes, ncols=3, frameon=False, cmap=magma)

In [ ]:
sc.pl.umap(adata_vasc[adata_vasc.obs['batch'].isin(['WT1', 'WT2'])], color=['batch', 'subtype',] + list_genes, ncols=4, frameon=False, cmap=magma, s=45)
sc.pl.umap(adata_vasc[adata_vasc.obs['batch'].isin(['KO1', 'KO2'])], color=['batch', 'subtype',] + list_genes, ncols=4, frameon=False, cmap=magma, s=45)

In [ ]:
for gene in list_genes: 
    stat_annot_gene(gene, adata_vasc, dict_vasc, type_plot='violin', add_stats=True)

## Heatmaps of relevant pathways

In [ ]:
cellular_response_hypoxia = ['Acaa2', 'Adam8', 'Ado', 'Adrb2', 'Ajuba', 'Ak4', 'Akt1', 'Aqp1', 'Aqp3', 'Atf2', 'Bad', 'Bcl2', 'Bmyc', 'Bnip3', 'Bnip3l', 'Cbs', 'Cd34', 'Cited2', 'Commd1', 'Cpeb1', 'Cpeb2', 'Cr1l', 'Ddah1', 'Egln1', 'Egln2', 'Egln3', 'Egr1', 'Eif4ebp1', 'Eno1', 'Ep300', 'Epas1', 'Epha4', 'Fam162a', 'Fmn2', 'Fndc1', 'Gata6', 'Gnb1', 'Hif1a', 'Hif1an', 'Hif3a', 'Higd1a', 'Hp1bp3', 'Hyou1', 'Kcnk3', 'Kdm6a', 'Map2k1', 'Mgarp', 'Mlst8', 'Mtor', 'Myc', 'Ndnf', 'Nfe2l2', 'Nop53', 'Notch1', 'Npepps', 'Ogt', 'P4hb', 'Pdk1', 'Pdk3', 'Pgk1', 'Pik3cb', 'Pink1', 'Plk3', 'Ppard', 'Pparg', 'Prkce', 'Pten', 'Ptgis', 'Rbpj', 'Rgcc', 'Rock2', 'Rora', 'Rptor', 'Rtn4', 'Scn2a', 'Sdhd', 'Sirt1', 'Sirt2', 'Slc2a4', 'Slc8a3', 'Slc9a1', 'Stox1', 'Stub1', 'Suv39h1', 'Suv39h2', 'Tbl2', 'Tert', 'Tigar', 'Tmbim6', 'Trp53', 'Twist1', 'Ubqln1', 'Usp19', 'Vasn', 'Vegfa', 'Vhl', 'Vldlr', 'Zfas1', 'Zfp36l1']

cellular_response_hypoxia_filtered = ['Adam8', 'Adrb2', 'Ak4', 'Akt1', 'Aqp1', 'Atf2', 'Bad', 'Bmyc', 'Bnip3', 'Bnip3l', 
 'Cd34', 'Cited2', 'Commd1', 'Cpeb1', 'Cr1l', 'Egln2', 'Egln3', 'Egr1', 'Eif4ebp1', 'Eno1', 
 'Epas1', 'Fam162a', 'Fndc1', 'Gnb1', 'Hif1a', 'Higd1a', 'Hp1bp3', 'Hyou1', 
 'Nfe2l2', 'Nop53', 'Npepps', 'Ogt', 'P4hb', 'Pgk1', 'Rbpj', 'Rgcc', 'Rora', 'Rtn4', 'Sdhd', 
 'Sirt1', 'Sirt2',  'Tmbim6', 'Trp53', 'Twist1', 'Ubqln1', 'Usp19', 'Vasn', 'Vegfa', 'Vhl', 'Vldlr', 'Zfas1', 'Zfp36l1']
# GO:0071456



vascular_endothelial_GF_production = ['Adgrg1', 'Adora2b', 'Arnt', 'Atf4', 'Brca1', 'Bsg', 'C3', 'C5ar1', 'Ccl2', 'Ccr2', 'Cxcl17', 'Cyp1b1', 'Eif2ak3', 'Flt1', 'Flt4', 'Hif1a', 'Hpse', 'Il1b', 'Il6', 'Il6ra', 'Ndrg2', 'Nodal', 'Nutf2', 'Ptgs2', 'Rora', 'Sars', 'Sulf1', 'Sulf2', 'Tnf']

vascular_endothelial_GF_production_filtered = ['Adgrg1', 'Atf4', 'Bsg', 'C3', 'Ccl2', 'Ccr2', 'Flt1', 'Flt4', 'Hif1a', 
                                               'Il1b', 'Il6', 'Ndrg2', 'Ptgs2', 'Rora', 'Sars', 'Sulf1']
# GO:0010573



blood_vessel_endothelial_cell_migration = ['Abl1', 'Acvrl1', 'Adam8', 'Adam17', 'Akt1', 'Akt3', 'Alox12', 'Amot', 'Amotl1', 'Angpt1', 'Angpt2', 'Angpt4', 'Anxa1', 'Apoa1', 'Apoe', 'Atp2b4', 'Atp5a1', 'Atp5b', 'Card10', 'Cd40', 'Cdh5', 'Cib1', 'Cln3', 'Col18a1', 'Csnk2b', 'Cxcl12', 'Cxcr4', 'Cyp1b1', 'Dll4', 'Efna1', 'Efnb2', 'Egr3', 'Emp2', 'Epha2', 'Ephb4', 'Ets1', 'Fbxw7', 'Fgf2', 'Fgf18', 'Fgfr1', 'Foxc2', 'Gadd45a', 'Gata2', 'Gpld1', 'Gpx1', 'Grem1', 'Hdac5', 'Hdac7', 'Hdac9', 'Hif1a', 'Hmgb1', 'Hmox1', 'Hspb1', 'Igf1', 'Igf2', 'Itgb1', 'Itgb1bp1', 'Jcad', 'Jup', 'Kdr', 'Klf4', 'Lemd3', 'Map2k3', 'Map2k5', 'Map3k3', 'Mecp2', 'Mef2c', 'Meox2', 'Mia3', 'Mmrn2', 'Myh9', 'Nf1', 'Nfe2l2', 'Nos3', 'Notch1', 'Nr4a1', 'Nrp1', 'Nus1', 'P2rx4', 'Pdcd10', 'Pdgfb', 'Pik3c2a', 'Pik3r2', 'Pik3r3', 'Plcg1', 'Plk2', 'Pparg', 'Prcp', 'Prkca', 'Prkd1', 'Prkd2', 'Prl7d1', 'Ptgs2', 'Ptk2b', 'Rgcc', 'Rhoa', 'Rhoj', 'Robo1', 'Robo4', 'Scarb1', 'Sh3bp1', 'Sirt1', 'Slit2', 'Sp1', 'Spred1', 'Srf', 'Srpx2', 'Stard13', 'Stat5a', 'Tbxa2r', 'Tgfb1', 'Thbs1', 'Tnf', 'Vash1', 'Vegfa', 'Vegfc', 'Vhl']

blood_vessel_endothelial_cell_migration_filtered = ['Acvrl1', 'Adam17', 'Akt1', 'Amotl1', 'Angpt2', 'Anxa1', 
 'Apoe', 'Atp2b4', 'Atp5a1', 'Atp5b', 'Cdh5', 'Col18a1', 'Csnk2b', 'Cxcl12', 'Cxcr4', 
 'Dll4', 'Efna1', 'Efnb2', 'Emp2', 'Epha2', 'Ets1', 'Fgfr1', 'Gadd45a', 
 'Gata2', 'Gpx1', 'Grem1', 'Hdac5', 'Hdac7', 'Hif1a', 'Hmgb1', 'Hmox1', 'Hspb1', 'Igf2', 'Itgb1', 
 'Jcad', 'Jup', 'Kdr', 'Klf4', 'Map2k3', 'Mef2c', 'Meox2', 'Mia3', 'Mmrn2', 
 'Myh9', 'Nfe2l2', 'Nr4a1', 'Nrp1', 'Pdcd10', 'Pdgfb', 'Pik3c2a', 
 'Plcg1', 'Plk2', 'Prcp', 'Prkd2', 'Ptgs2', 'Rgcc', 'Rhoa', 'Rhoj', 
 'Robo4', 'Scarb1', 'Sirt1', 'Slit2', 'Sp1', 'Stard13', 
 'Tgfb1', 'Thbs1', 'Vegfa']
# GO:0043534




endothelial_cell_activation = ['Acvrl1', 'Bmper', 'Cxcl10', 'Foxp1', 'Hoxa9', 'Ppia', 'Prmt5', 'Smad4', 'Socs5', 'Tcf4', 'Tcim']

endothelial_cell_activation_filtered = ['Acvrl1', 'Cxcl10', 'Foxp1', 'Ppia', 'Tcf4']
# GO:0042118 



sprouting_angiogenesis = ['Abl1', 'Adamts9', 'Adgra2', 'Agtr1a', 'Akt1', 'Akt3', 'Angpt1', 'Anxa1', 'Aplnr', 'Bmp4', 'Bmper', 'Card10', 'Cdc42', 'Cdh13', 'Ceacam1', 'Cib1', 'Creb3l1', 'Dll1', 'Dll4', 'E2f2', 'E2f7', 'E2f8', 'Efnb2', 'Egr3', 'Eng', 'Epha2', 'Ephb4', 'Epn1', 'Epn2', 'Esm1', 'Fbxw7', 'Fgf1', 'Fgf2', 'Flt1', 'Flt4', 'Foxc2', 'Gata2', 'Ghrl', 'Glul', 'Gpld1', 'Grem1', 'Hdac5', 'Hdac7', 'Hdac9', 'Hmgb1', 'Hmox1', 'Il10', 'Il12a', 'Itga5', 'Itgb1', 'Itgb1bp1', 'Jak1', 'Jcad', 'Jmjd6', 'Jmjd8', 'Kdr', 'Klf2', 'Klf4', 'Lef1', 'Loxl2', 'Map2k5', 'Map3k3', 'Meox2', 'Mia3', 'Mmrn2', 'Naxe', 'Ngfr', 'Notch1', 'Nr4a1', 'Nrarp', 'Nrp1', 'Otulin', 'Parva', 'Pdcd10', 'Pgf', 'Pik3c2a', 'Pik3cb', 'Pik3r2', 'Pik3r3', 'Pkm', 'Plk2', 'Ppp1r16b', 'Ptgs2', 'Ptk2b', 'Ramp2', 'Reck', 'Rhoa', 'Rhoj', 'Rnf213', 'Robo1', 'Rspo3', 'Rtn4', 'S100a1', 'Sema3e', 'Sema5a', 'Sema6a', 'Sh2b3', 'Slit2', 'Smad1', 'Spred1', 'Srf', 'Srpx2', 'Stard13', 'Synj2bp', 'Tbxa2r', 'Tek', 'Thbs1', 'Tjp1', 'Tnn', 'Vegfa', 'Vegfb', 'Vegfc', 'Vegfd', 'Vstm4']

sprouting_angiogenesis_filtered = ['Adamts9', 'Agtr1a', 'Akt1', 'Akt3', 'Anxa1', 'Aplnr', 'Bmp4', 'Card10', 'Cdc42', 'Cdh13', 
 'Cib1', 'Creb3l1', 'Dll4', 'Efnb2', 'Eng', 'Epha2', 'Ephb4', 'Epn1', 'Epn2', 
 'Esm1', 'Flt1', 'Flt4', 'Foxc2', 'Gata2', 'Glul', 'Hdac5', 'Hdac7', 
 'Hmgb1', 'Hmox1', 'Itga5', 'Itgb1', 'Itgb1bp1', 'Jak1', 'Jcad', 'Jmjd6', 'Kdr', 'Klf2', 'Klf4', 
 'Loxl2', 'Map3k3', 'Meox2', 'Mia3', 'Mmrn2', 'Nr4a1', 'Nrarp', 'Nrp1', 'Otulin', 'Parva', 
 'Pdcd10', 'Pgf', 'Pik3c2a', 'Pkm', 'Plk2', 'Ptgs2', 'Ramp2', 'Rhoa', 
 'Rhoj', 'Rnf213', 'Rtn4', 'Sema5a', 'Sema6a', 'Sh2b3', 'Smad1', 
 'Stard13', 'Synj2bp', 'Tek', 'Thbs1', 'Tjp1', 'Vegfa', 'Vegfb']
# GO:0002040




positive_regulation_angiogenesis = ['Abl1', 'Acvrl1', 'Adam12', 'Add1', 'Adm', 'Adm2', 'Aggf1', 'Ago2', 'Agtr1a', 'Akt3', 'Angpt2', 'Angpt4', 'Anxa3', 'Aplnr', 'Aqp1', 'Bmper', 'Brca1', 'Btg1', 'C3', 'C5ar1', 'Camp', 'Ccl5', 'Ccl11', 'Cd34', 'Cd40', 'Cdh5', 'Chil1', 'Cma1', 'Ctsh', 'Cx3cl1', 'Cx3cr1', 'Cxcr2', 'Cxcr3', 'Cxcr4', 'Cyp1b1', 'Cysltr1', 'Cysltr2', 'Ddah1', 'Dll1', 'Ecm1', 'Emc10', 'Emilin1', 'Emilin2', 'Emp2', 'Eng', 'Epha1', 'Erap1', 'Ets1', 'F3', 'Fgf1', 'Fgf2', 'Fgf18', 'Flt1', 'Foxc2', 'Gab1', 'Gata2', 'Gata6', 'Ghrl', 'Grem1', 'Grn', 'Hgf', 'Hif1a', 'Hipk2', 'Hk2', 'Hmga2', 'Hmgb1', 'Hmox1', 'Hpse', 'Hspb1', 'Hspb6', 'Hyal1', 'Igf2', 'Il1b', 'Il10', 'Itga5', 'Itgb1', 'Itgb2', 'Itgb3', 'Itgb8', 'Jak1', 'Jcad', 'Jmjd8', 'Jup', 'Kdr', 'Klf4', 'Lgals3', 'Lrg1', 'Mdk', 'Mmp9', 'Mtdh', 'Mydgf', 'Nfe2l2', 'Ninj1', 'Nodal', 'Nos3', 'Notch4', 'Nras', 'Nrp1', 'Pak4', 'Pdcd6', 'Pdcl3', 'Pgf', 'Pik3cd', 'Pik3r6', 'Pkm', 'Plcg1', 'Prkca', 'Prkcb', 'Prkd1', 'Prkd2', 'Ptgis', 'Ptk2b', 'Pxn', 'Ramp2', 'Rapgef3', 'Rhob', 'Rras', 'Rtn4', 'Runx1', 'S100a1', 'Sash1', 'Sema5a', 'Serpine1', 'Sirt1', 'Sirt6', 'Smad1', 'Smoc2', 'Sp1', 'Sphk1', 'Stat3', 'Stim1', 'Tbxa2r', 'Tek', 'Tert', 'Tgfbr2', 'Thbs1', 'Tie1', 'Tjp1', 'Tlr3', 'Tnfrsf1a', 'Tnn', 'Vash2', 'Vegfa', 'Vegfb', 'Vegfc', 'Vegfd', 'Wars2', 'Wnt5a', 'Xbp1', 'Zc3h12a']

positive_regulation_angiogenesis_filtered = ['Acvrl1', 'Add1', 'Adm', 'Aggf1', 'Ago2', 'Agtr1a', 'Akt3', 'Anxa3', 'Aplnr', 'Aqp1', 
 'Btg1', 'C3', 'Camp', 'Ccl11', 'Cd34', 'Cdh5', 'Ctsh', 'Cx3cl1',  'Cxcr4', 'Ddah1', 'Dll1', 'Ecm1', 'Emc10', 'Emilin2', 
 'Emp2', 'Eng', 'Ets1', 'Flt1', 'Gab1', 'Gata2', 'Gata6',  'Grn', 'Hgf', 'Hif1a', 'Hipk2', 'Hk2', 'Hmgb1', 'Hmox1', 'Hspb1', 'Il1b', 
 'Itga5', 'Itgb1', 'Itgb2', 'Itgb3', 'Itgb8', 'Jak1', 'Jcad', 'Jup', 'Kdr', 'Klf4', 'Lgals3', 'Lrg1', 'Mdk', 
 'Mtdh', 'Nfe2l2', 'Ninj1', 'Nos3', 'Notch4', 'Nras', 'Nrp1', 'Pdcd6', 'Pdcl3', 'Pgf', 
 'Pkm', 'Prkd2', 'Pxn', 'Ramp2', 'Rhob',  'Rras', 'Rtn4', 'Runx1', 'Sash1', 'Serpine1', 'Sirt1', 'Smad1', 'Sp1', 
 'Stat3', 'Tek', 'Thbs1', 'Tie1', 'Tjp1', 'Tnfrsf1a', 'Vegfa',  'Vegfb', 'Wnt5a', 'Xbp1', 'Zc3h12a']
# GO:0045766



inflammatory_response_wounding = ['Abi3bp', 'Ager', 'Ccr2', 'Cd24a', 'Cd44', 'Cfh', 'F2r', 'Git1', 'Grn', 'Hif1a', 'Hmox1', 'Mdk', 'Pparg', 'Timp1', 'Tlr4', 'Wdr83']

inflammatory_response_wounding_filtered = ['Abi3bp', 'Cd24a', 'Cd44', 'F2r', 'Git1', 'Grn', 'Hif1a', 'Hmox1', 'Mdk', 'Pparg', 'Timp1']
# GO:0090594

In [ ]:
sc.pl.matrixplot(adata_vasc, groupby='subtype', var_names=cellular_response_hypoxia_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 11), vmax=0.85, categories_order=list(dict_vasc.keys()))

In [ ]:
sc.pl.matrixplot(adata_vasc, groupby='subtype', var_names=vascular_endothelial_GF_production_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 6), vmax=0.85, categories_order=list(dict_vasc.keys()))

In [ ]:
sc.pl.matrixplot(adata_vasc, groupby='subtype', var_names=blood_vessel_endothelial_cell_migration_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 14), vmax=0.85, categories_order=list(dict_vasc.keys()))

In [ ]:
sc.pl.matrixplot(adata_vasc, groupby='subtype', var_names=endothelial_cell_activation_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 2), vmax=0.85, categories_order=list(dict_vasc.keys()))

In [ ]:
sc.pl.matrixplot(adata_vasc, groupby='subtype', var_names=positive_regulation_angiogenesis_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 14), vmax=0.85, categories_order=list(dict_vasc.keys()))

In [ ]:
sc.pl.matrixplot(adata_vasc, groupby='subtype', var_names=inflammatory_response_wounding_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 3), vmax=0.85, categories_order=list(dict_vasc.keys()))

In [ ]:
adata_vasc.write_h5ad('adatas/adata_vasculature.h5')